<a href="https://colab.research.google.com/github/NeuralClassifier/ReusabilityRate_ML/blob/ClassificationModelsPred/ClassifierPred_Reusability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-learn==1.0.1
!pip install -U tensorflow==2.7.0
!pip install -U seaborn==0.11.2
!pip install -U matplotlib==3.5.0
!pip install -U pandas==1.3.4

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import KFold 
import statistics

import warnings


In [3]:
class_df = pd.read_csv('DataSets/cls_normalized.csv')
class_df.drop(['Unnamed: 0'],axis = 1, inplace=True)
class_df

,Project,LongName,Name,LCOM5,NL,NLE,WMC,CBO,CBOI,NII,NOI,RFC,AD,CD,TCD,CLOC,TCLOC,DLOC,PDA,DIT,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,ReuseRate,ReuseRate_lbl
0,ch.qos.logback/logback-classic,ch.qos.logback.classic.AsyncAppender,AsyncAppender,-0.976331,-0.916667,-0.8,-0.990539,-0.982979,-1.000000,-1.000000,-0.984127,-0.979975,-0.333334,-0.110417,-0.110417,-0.995522,-0.995522,-0.995389,-0.993651,-1.000000,-0.991793,-0.993978,-0.993978,-1.000000,-0.995344,-0.997199,-1.0000,-0.995408,-0.996554,-0.997253,5.307617,4
1,ch.qos.logback/logback-classic,ch.qos.logback.classic.BasicConfigurator,BasicConfigurator,-0.988166,-1.000000,-1.0,-0.996216,-0.957447,-0.996370,-0.998538,-0.988095,-0.987484,-0.333334,-0.322223,-0.322223,-0.996802,-0.996802,-0.997695,-0.993651,-0.777778,-0.990828,-0.993548,-0.993548,-1.000000,-0.994796,-1.000000,-1.0000,-0.997704,-0.993109,-0.997253,4.000000,3
2,ch.qos.logback/logback-classic,ch.qos.logback.classic.ClassicConstants,ClassicConstants,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-0.974592,-1.000000,-1.000000,-1.000000,-1.000000,-0.346428,-0.346428,-0.997121,-0.997121,-0.997036,-1.000000,-1.000000,-0.991311,-0.992687,-0.992687,-1.000000,-0.995070,-0.952381,-1.0000,-1.000000,-1.000000,-1.000000,5.307617,4
3,ch.qos.logback/logback-classic,ch.qos.logback.classic.Level,Level,-0.988166,-0.916667,-0.8,-0.912961,-1.000000,-0.883848,-0.970760,-1.000000,-0.967459,1.000000,-0.242484,-0.242484,-0.969615,-0.969615,-0.969043,-0.923810,-1.000000,-0.923244,-0.941284,-0.941284,-1.000000,-0.956450,-0.932773,-1.0000,-0.985075,-0.966527,-0.984890,5.307617,4
4,ch.qos.logback/logback-classic,ch.qos.logback.classic.Logger,Logger,-0.988166,-0.750000,-0.4,-0.708609,-0.957447,-0.916515,-0.969298,-0.956349,-0.732165,-0.851852,-0.581752,-0.581752,-0.958420,-0.958420,-0.960481,-0.961905,-1.000000,-0.758146,-0.838047,-0.838047,-0.929293,-0.862777,-0.971989,-0.9375,-0.889782,-0.877923,-0.890110,5.307617,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24789,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$4,SecuritySupport$4,-0.988166,-1.000000,-1.0,-0.998108,-1.000000,-1.000000,-1.000000,-1.000000,-0.997497,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.998069,-0.999140,-0.999140,-1.000000,-0.998904,-1.000000,-1.0000,-0.998852,-0.999508,-0.998626,0.000000,0
24790,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$5,SecuritySupport$5,-0.988166,-1.000000,-1.0,-0.998108,-1.000000,-1.000000,-1.000000,-1.000000,-0.997497,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.998069,-0.999140,-0.999140,-1.000000,-0.998904,-1.000000,-1.0000,-0.998852,-0.999508,-0.998626,0.000000,0
24791,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$6,SecuritySupport$6,-0.988166,-0.916667,-0.8,-0.996216,-1.000000,-1.000000,-1.000000,-1.000000,-0.997497,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.995173,-0.997849,-0.997849,-1.000000,-0.997261,-1.000000,-1.0000,-0.998852,-0.997539,-0.998626,0.000000,0
24792,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$7,SecuritySupport$7,-0.988166,-1.000000,-1.0,-0.996216,-1.000000,-1.000000,-1.000000,-1.000000,-0.997497,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.998069,-0.999140,-0.999140,-1.000000,-0.998904,-1.000000,-1.0000,-0.998852,-0.999508,-0.998626,0.000000,0


In [4]:
package_df = pd.read_csv('DataSets/pkg_normalized.csv')
package_df.drop(['Unnamed: 0'],axis = 1, inplace=True)
package_df

,Project,LongName,Name,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL,ReuseRate,ReuseRate_lbl
0,ch.qos.logback/logback-classic,<root_package>,<root_package>,-1.000000,-1.000000,-0.255446,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956749,-1.000000,-0.959630,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000,0.0,0
1,ch.qos.logback/logback-classic,ch,ch,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000,0.0,0
2,ch.qos.logback/logback-classic,unnamed package,unnamed package,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.999661,-0.999712,-0.999885,-1.000000,-0.999838,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.0,0
3,ch.qos.logback/logback-classic,ch.qos,qos,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000,0.0,0
4,ch.qos.logback/logback-classic,ch.qos.logback,logback,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,xerces/xercesImpl,org.apache.xerces.impl.xs.identity,identity,0.968254,0.033413,0.083579,-0.977705,-0.993081,-0.931073,-0.989014,-0.992390,-0.987862,-0.995148,-0.940460,-0.996360,-0.991098,-0.995164,-0.996647,-0.993460,-0.996841,-0.977604,0.0,0
2189,xerces/xercesImpl,org.apache.xerces.impl.xs.models,models,0.402986,-0.011642,0.036337,-0.964447,-0.988967,-0.947749,-0.991672,-0.986761,-0.979412,-0.991769,-0.953992,-0.993669,-0.988131,-0.996263,-0.995377,-0.987849,-0.996484,-0.977604,0.0,0
2190,xerces/xercesImpl,org.apache.xerces.impl.xs.opti,opti,0.081272,-0.000149,0.048387,-0.940306,-0.981476,-0.829906,-0.972889,-0.978261,-0.965010,-0.986012,-0.745602,-0.989603,-0.978087,-0.979338,-0.984151,-0.985873,-0.983313,-0.966405,3.0,2
2191,xerces/xercesImpl,org.apache.xerces.impl.xs.traversers,traversers,0.400000,-0.532498,-0.509803,-0.920482,-0.975324,-0.984436,-0.997519,-0.907371,-0.904436,-0.961795,-0.907984,-0.955701,-0.924218,-0.992527,-0.985268,-0.919689,-0.996603,-0.935050,0.0,0


In [5]:
package_df.iloc[:,3:-2]

,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL
0,-1.000000,-1.000000,-0.255446,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956749,-1.000000,-0.959630,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000
1,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000
2,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.999661,-0.999712,-0.999885,-1.000000,-0.999838,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
3,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000
4,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,0.968254,0.033413,0.083579,-0.977705,-0.993081,-0.931073,-0.989014,-0.992390,-0.987862,-0.995148,-0.940460,-0.996360,-0.991098,-0.995164,-0.996647,-0.993460,-0.996841,-0.977604
2189,0.402986,-0.011642,0.036337,-0.964447,-0.988967,-0.947749,-0.991672,-0.986761,-0.979412,-0.991769,-0.953992,-0.993669,-0.988131,-0.996263,-0.995377,-0.987849,-0.996484,-0.977604
2190,0.081272,-0.000149,0.048387,-0.940306,-0.981476,-0.829906,-0.972889,-0.978261,-0.965010,-0.986012,-0.745602,-0.989603,-0.978087,-0.979338,-0.984151,-0.985873,-0.983313,-0.966405
2191,0.400000,-0.532498,-0.509803,-0.920482,-0.975324,-0.984436,-0.997519,-0.907371,-0.904436,-0.961795,-0.907984,-0.955701,-0.924218,-0.992527,-0.985268,-0.919689,-0.996603,-0.935050


In [7]:
warnings.filterwarnings('ignore')
class ReuseTradML:
  def __init__(self,level):
    self.ml_algo_list = ['GNB',
                         'LRC',
                         'KNN',
                         'DTC',
                         'RFC',
                         'BNB',
                         'SVC',
                         'MLPC']

    self.metrics = ['Accuracy',
                    'Precision',
                    'Recall']
    self.data_columns = None
    self.level=level

    if self.level == 'package':
      #target_scaler = MinMaxScaler()
      data = pd.read_csv('DataSets/pkg_normalized.csv')
      data.drop(['Unnamed: 0'],axis=1,inplace=True)
      df = data.iloc[:,3:-2]
      self.data_columns = df.columns
      self.X_np = np.array(df)
      #target_scaler = MinMaxScaler()
      #target_scaler.fit(self.X_np)
      #self.X = target_scaler.transform(self.X_np)

      self.Y = data.iloc[:,-1]
      #self.Y = self.Y.values.reshape(-1, 1)
      #target_scaler = MinMaxScaler()
      #target_scaler.fit(self.Y)
      #self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif self.level == 'class':
      #target_scaler = MinMaxScaler()
      data = pd.read_csv('DataSets/cls_normalized.csv')
      data.drop(['Unnamed: 0'],axis=1,inplace=True)
      df = data.iloc[:,3:-2]
      self.data_columns = df.columns
      self.X_np = np.array(df)
      #target_scaler = MinMaxScaler()
      #target_scaler.fit(self.X_np)
      #self.X = target_scaler.transform(self.X_np)

      self.Y = data.iloc[:,-1]
      #self.Y = self.Y.values.reshape(-1, 1)
      #target_scaler = MinMaxScaler()
      #target_scaler.fit(self.Y)
      #self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif self.level == 'project':
      self.X_np = np.array(data.iloc[:,4:])
      self.other = data.iloc[:,:4]

    else:
      print('Error, Please try: class, package, or project. Thanks!')
      self.X=None
      self.Y=None
      self.other=None

  def apply_ml(self):
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
  
    metric = []
    metric_count = 0

    level = []
    model = []

    mean_acc = []
    stdv_acc = []
    mean_pre = []
    stdv_pre = []
    mean_rec = []
    stdv_rec = []
    

    for i in range(self.X_np.shape[1]):
      #print(self.X[i])
      for j in self.ml_algo_list:
          
        warnings.filterwarnings('ignore')
        #['Polynomial Regression','MLPRegressor','BayesianRidge','SGD Regressor','XGB Regressor','Decision Tree Regressor','ElasticNet','SVM Regressor',]
        if j == 'GNB':
          clss_model = GaussianNB()
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          model.append(j)

        #if j == 'Logistic Regression':
          #reg = LogisticRegression()
          #print('Metric: ',self.data_columns[i])
          #print('Level: ',self.level)
          #print('Model: ',j)

        if j == 'LRC':
          clss_model = LogisticRegression()
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          model.append(j)

        if j == 'KNN':
          clss_model = KNeighborsClassifier()
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          model.append(j)

        if j == 'DTC':
          clss_model = DecisionTreeClassifier()
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          model.append(j)

        if j == 'RFC':
          clss_model = RandomForestClassifier()
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          model.append(j)

        if j == 'BNB':
          clss_model = BernoulliNB()
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          model.append(j)

        if j == 'SVC':
          clss_model = SVC()
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          model.append(j)
        
        if j == 'MLPC':
          clss_model = MLPClassifier()
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          model.append(j)
        
        print('At i: ',i,' x = ',self.X_np[:,i])
        print()
        #temp_data = self.X_np[:,i].reshape(-1, 1)
        #target_scaler = MinMaxScaler()
        #target_scaler.fit(temp_data)
        #temp_data = target_scaler.transform(temp_data)
        level.append(self.level)
        metric.append(self.data_columns[i])

        cnt=0
        acc=[]
        pre=[]
        rec=[]
        for train_index, test_index in kf.split(self.X_np):
          cnt = cnt+1
          #print('Fold: ',cnt)
          X_train, X_test = self.X_np[train_index], self.X_np[test_index]
          y_train, y_test = self.Y[train_index], self.Y[test_index]

          clss_model.fit(X_train,y_train)
          y_pred = clss_model.predict(X_test)
          acc.append(accuracy_score(y_test, y_pred))
          pre.append(precision_score(y_test,y_pred,average='weighted'))
          rec.append(recall_score(y_test,y_pred,average='weighted'))

        mean_acc.append(statistics.mean(acc))
        stdv_acc.append(statistics.pstdev(acc))
        mean_pre.append(statistics.mean(pre))
        stdv_pre.append(statistics.pstdev(pre))
        mean_rec.append(statistics.mean(rec))
        stdv_rec.append(statistics.pstdev(rec))
    

      

    #break
    metric = pd.DataFrame(metric,columns=['Metric'])
    level = pd.DataFrame(level,columns=['Level'])
    model = pd.DataFrame(model,columns=['Model'])
    print()

    mean_acc = pd.DataFrame(mean_acc,columns=['Acc'])
    stdv_acc = pd.DataFrame(stdv_acc,columns=['STD Acc'])
    mean_pre = pd.DataFrame(mean_pre,columns=['Pre'])
    stdv_pre = pd.DataFrame(stdv_pre,columns=['STD Pre'])
    mean_rec = pd.DataFrame(mean_rec,columns=['Rec'])
    stdv_rec = pd.DataFrame(stdv_rec,columns=['STD Rec'])


    res_df = pd.concat([metric,level,model,mean_acc,stdv_acc,mean_pre,stdv_pre,mean_rec,stdv_rec],axis=1)

    return res_df

In [ ]:
rtml=ReuseTradML('class')
redf=rtml.apply_ml()
redf

Metric:  LCOM5
Level:  class
Model:  GNB

**************************************************************

At i:  0  x =  [-0.97633136 -0.98816568 -1.         ... -0.98816568 -0.98816568
 -0.98816568]

Metric:  LCOM5
Level:  class
Model:  LRC

**************************************************************

At i:  0  x =  [-0.97633136 -0.98816568 -1.         ... -0.98816568 -0.98816568
 -0.98816568]

Metric:  LCOM5
Level:  class
Model:  KNN

**************************************************************

At i:  0  x =  [-0.97633136 -0.98816568 -1.         ... -0.98816568 -0.98816568
 -0.98816568]

